### We are going to build a very trivial langgraph application. An application which just does the five steps we explained in 6_README.txt, i.e.
### Define a State class, create a graph builder, create nodes and egdes, compiler and finally run the graph.

### the nodes wont even do anything really agentic. :)

In [7]:
from langgraph.graph import StateGraph, START, END  # StateGraph is the graph_builder object, START and END are used to create when creating the starting and ending node of your workflow.
from langgraph.graph.message import add_messages # add_message is a very trivial reducer which takes two lists and concatenates the two lists.
import gradio as gr
from dotenv import load_dotenv
import os
from pydantic import BaseModel, Field
import random
from IPython.display import Image, display, Markdown
from typing import Annotated
from langchain_openai import ChatOpenAI

In [2]:
# We create a list of nounes and adjectives and when we create the node later on, it will randomly pick one noun and one adjective and return a new State object with the new noun and adjective.
nouns = [
    "mountain", "river", "forest", "computer", "ocean",
    "book", "planet", "castle", "robot", "guitar",
    "bridge", "car"
]

# List of random adjectives
adjectives = [
    "ancient", "massive", "silent", "glowing", "fragile",
    "noisy", "colorful", "mysterious", "swift", "brilliant",
    "dark", "icy"
]

In [ ]:
load_dotenv(override=True)

### Side note on how langgraph uses "Annotated" to install reducers. 
### You already know about python type hints that you can apply to arguments or function return type etc. Example:

```python
def shout(text: str) -> None:
    print(text.upper())
```

### Another way of specifying type hints is using the "Annotated" syntax.
```python
def shout(text: Annotated[str, "A text to print in upper case"]) -> None:
    print(text.upper())
```

### Annotated description inside the Annotated expression is not even parsed by the python interpreter. It simply discards it. However, langgraph uses it.
### the Annotated expression is used to specify the reducer. and we will use the "add_messages" reducer next.

### the add_messages reducer is a built-in reducer function that’s primarily used in chat-style applications. Its job is to accumulate (append) the messages exchanged between the user and the system over the course of the graph's execution. It adds messages returned by each node (usually in OpenAI-style {"role": ..., "content": ...} format) to the messages list in the graph's state. This helps maintain a chat history or conversation memory.

In [14]:
# Lets create the State class next. It could either by a pydantic class or a typed Dict. We will use pydantic.
class State(BaseModel):
    messages: Annotated[list, add_messages]


In [15]:
# Create a graph builder object.
graph_builder = StateGraph(State)


In [16]:
# create a node function. Remember that the node function takes a state and returns a state. and the state is immutable.
def our_first_node(old_state: State) -> State:
    reply = f"{random.choice(nouns)} are {random.choice(adjectives)}"
    # we create a json object, similar to openAI messages.
    messages = [
        {"role": "assistant", "content": reply}
    ]
    return State(messages=messages)

# create a node.
node = graph_builder.add_node("first_node", our_first_node)


In [17]:
# create edges with just the single nodes.
graph_builder.add_edge(START, "first_node")
graph_builder.add_edge("first_node", END)

# compile the graph.
graph = graph_builder.compile()

In [ ]:
# Display the graph.
display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
# Finally create the gradio chat interface. Remember that the chat interface takes a current message and the history.
# In the chat, we want to invoke our graph as it is already compiled and ready to use. We really dont make any LLM call just yet.
def chat(user_input: str, history):
    message = {"role": "user", "content": user_input} # this is the message we will give to the graph.
    messages = [message]
    state = State(messages=messages)
    result = graph.invoke(state)
    print(result)
    return result["messages"][-1].content

gr.ChatInterface(chat, type="messages").launch()

# What really is happening:
# 1.A user enters a message.
# 2.The message is added to the state.
# 3.The state is passed to the graph.
# 4.The graph invokes the node function.
# 5.The node function returns a state.
# 6.The reducer function(s) is called with the old state and the new state.
# 7.The reducer function(s) update the state based on the node function's output.
# 8.The updated state is passed to the next node function.
# 9.This continues until the graph reaches the END node.
# 10.The final state is returned to the user.

### If you really see, langgraph is all about python functions - it doesnt need to invoke LLMs!!
### Next, lets do all steps above again, but this time we will use a LLM.

In [4]:
class State(BaseModel):
    messages: Annotated[list, add_messages]

In [5]:
graph_builder = StateGraph(State)

In [ ]:
llm = ChatOpenAI(model="gpt-4o-mini")

def chatbot_node(old_state: State) -> State:
    response = llm.invoke(old_state.messages)
    new_state = State(messages=[response])
    return new_state

graph_builder.add_node("chatbot_node", chatbot_node)
graph_builder.add_edge(START, "chatbot_node")
graph_builder.add_edge("chatbot_node", END)

graph = graph_builder.compile()
display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
def chat(user_input: str, history):
    message = {"role": "user", "content": user_input} # this is the message we will give to the graph.
    messages = [message]
    state = State(messages=messages)
    result = graph.invoke(state)
    print(result)
    return result["messages"][-1].content

gr.ChatInterface(chat, type="messages").launch()

# In your chat, tell the chat interface your name first and then ask the chatbot if it knows your name.
# It wont remember that as we are not using any memory. In fact the history argument is unused.